## TF-IDF and k-Means Clustering

We are using a dataset of Amazon reviews. It is compiled of three topics and contains ∼200k reviews. Convert those reviews into TF-IDF form, and use k-means clustering to analyze the results. Finally, we will check the resulting clusters and see how well-defined are the clusters.


The data is downloaded from http://jmcauley.ucsd.edu/data/amazon/ and contains 3 topics: 
- baby
- musical instruments
- beauty. 

Musical instruments is the smallest collection you won't see many of these when inspecting the results. The original json files are transformed into csv, and contain four variables, date, summary, review, and rating.

In [1]:
import pandas as pd
import numpy as np


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

## I. Load and clean the dataset

In [2]:
amreviews = pd.read_csv("amazon-reviews.csv.bz2", sep="\t")
amreviews.sample(10)
amreviews.shape

(205331, 4)

### Remove missing and empty reviews

In [3]:
#Check for na values
amreviews.review.isna().sum()

80

In [4]:
#Check for empty strings 
np.where(amreviews.review.apply(lambda x: x == ''))

(array([], dtype=int64),)

Based on above we only have nan values. Let's go ahead and drop this.

In [5]:
#drop na 
amreviews_mod = amreviews.drop(amreviews[amreviews.review.isna()].index).reset_index()

In [6]:
#dropping duplicates in case there are any
amreviews_mod[amreviews_mod.duplicated(subset=None, keep='first')][['review']].count()
amreviews_mod.drop_duplicates(keep = 'first', inplace = True) 

Taking a subset of the data

In [7]:
# take a sample of 5000 reviews
np.random.seed(1867)
amreviews_sample = amreviews_mod.sample(5000)

In [8]:
#view sample
amreviews_sample.sample(10)

,index,date,summary,review,rating
78644,78674,2011-05-16,love it now that i've gotten used to it!,"I had been searching everywhere for a car seat-- I absolutely HATED the cheap flimsy plastic ones that seem to be everywhere, and I fell in love with Peg because of the sleek design and great safety ratings. However, I was so anxious about this car seat after reading the negative reviews that we went to the Peg Perego outlet in Ft. Wayne, IN to see the item for ourselves before committing. We loved it! When we got the new items home, we could not figure out how to latch and unlatch the car seat, or how to attach it to the pliko switch stroller we bought. The instructions were poor, had no words (only pictures), and we were so frustrated. I was ready to take everything back and call it quits. However, once we played around with the seat a bit (this took a few hours and a lot of patience) and figured out how to latch/unlatch/adjust it, we realized how simple and easy the whole thing is! And as a plus, it fits so nicely with the pliko switch stroller (which we LOVE). I adore that this car seat fits up to 30lbs, as pretty much every other seat I found only went to 22lbs. The seat is heavier than other car seats, but it isn't cheap graco-quality plastic and actually feels like it has some durability to it. I have pathetic upper body strength and even I can carry it around without much of a problem. In any case, the seat isn't anywhere near heavy enough to warrant the serious doubts I had based on many of the reviews here. The product gets a 5 for quality and style, and a 3 for ease of instruction, which average to a 4. Also, if you live close to the outlet it is definitely worth a visit--- their prices are much lower and they offer the 2011 models plus have a good return policy.",4
93715,93752,2012-12-25,Kids love,"My toddler loves this toy. She can't turn the crank yet, but waits for the bear to pop up when someone else turns it. Then she hugs it and tries to &#34;free it&#34; from the box. It's a hoot! And she really just delights in it.",5
61100,61125,2010-09-04,"Absorbent, but they shrink.","WARNING: Do not use an enzyme spray on these prefolds, it will decrease the absorbancy.PROs: DurableCONs: Shrink in dryer, take too long to dry.",3
120860,120904,2012-10-10,Cheap and works everytime!,This is such a great and amazing buy for mommies to cut their little one's nails. I have been using this for 6 month now and NOT once did I hurt my baby. This nail clipper works wonderfully and the filing on the other end makes it so convenient. I would highly recommend this item for all mommies. Especially first time mommies that doesn't know how to cut their baby's nails.,5
162512,162572,2016-03-15,Perfect,Wanted a natural nail polish. And this is the closest anyone makes. I love how it makes my natural nails look beautiful. Don't have to worry when it starts chipping because you barely notice. Made my nails a lot less yellow. Love this product!,5
35418,35434,2011-01-11,Decent liners for protecting your diaper,"I purchased these because I wanted a liner that would help prevent staining on my expensive diapers. I wasn't expecting these to act as a doubler. (They definitely would NOT work for extra absorbency by the way.) They do the trick for keeping my diapers stain free and make messy clean-ups easier.They are a low-cost in comparison to other liners, so that is a plus. They're not as soft as I'd like them to be and mine shrank quite a bit, but they're a decent product for the price.",4
132678,132729,2013-02-19,Nice gate,"This thing must have gone to heck and back before it came to my doorstep. Two of the bars were bent during shipping, stripping the finish off the metal. One of the connecting thingies was broken so we had to leave off one of the panels. By some miracle we still got it on the wall. Since we did, I was not going to wait for a replacement piece. Anyway, we got it on, it looks great, we fixed the bent bars, and our kitchen is now 

Based on analysing the above 10 reviews, it seems that there is a category related to 'all things baby' where cribs, and breast pumps are being reviewed. The second category is electronics - where a musical instrument and a monitor was reviewed.

The last category is a tough one, but looking at - "love this color!! goes on well and lasts 5-7 days.", "Hydrates without weighing hair down.", it seems it could be related to hair products.

## II. TF-IDF Transform

### Transform reviews into Bag-Of-Words

In [9]:
# import
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words

In [10]:
# convert in BOW
vectorizer = CountVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS)
X = vectorizer.fit_transform(amreviews_sample.review.values)

### Write a function that performs TF-IDF

In [11]:
def get_tfidf_matrix(X):
    #function to calculate term freq for each element
    tf = lambda x: np.log(1+x)
    # apply tf lambda on matrix
    tf_result = np.apply_along_axis(tf,1,X.toarray())  
    #Get the total number of documents/reviews
    N = len(X.toarray())
    # Get frequency of each word across all documents
    overall_freq = X.toarray().sum(axis=0)
    # function to calculate idf for each word
    idf = lambda x: np.log(N/(1+x))
    idf_result = idf(overall_freq)
    #multiply tf with idf for each document
    return np.multiply(tf_result,idf_result)

In [13]:
#get tf-idf matrix for reviews
tf_idf = get_tfidf_matrix(X)

## III. Implement k-means clustering using cosine similarity

**Let's make 5 clusters!**

#### assign all reviews into arbitrary clusters

In [14]:
class Kmeans:

    def __init__(self, k, seed = None, max_iter = 200):
        self.k = k
        self.seed = seed
        if self.seed is not None:
            np.random.seed(self.seed)
        self.max_iter = max_iter
        
            
    # randomly Initialise Centroids
    def random_centres(self, data):
        initial_centres = np.random.permutation(data.shape[0])[:self.k]
        self.centres = data[initial_centres]

        return self.centres
    
    # get  distance of data from clusters and assign data point to closest cluster.
    def assign_clusters(self, data):
        if data.ndim == 1:
            data = data.reshape(-1, 1)
        dist_to_centres =  self.cosine_sim(data, self.centres)
        self.cluster_val = np.argmax(dist_to_centres, axis = 1)
        return  self.cluster_val
    
    # get the cosine similarity of two vectors
    def cosine_sim(self, X, Y):
        dot_product = np.dot(X, Y.T)
        review_norm = np.linalg.norm(X, axis = 1)
        review_norm = review_norm.reshape(X.shape[0],1)
        center_norm = np.linalg.norm(Y, axis = 1)
        center_norm = center_norm.reshape(Y.shape[0],1)
        return dot_product / (review_norm.dot(center_norm.T))
        
    # get average of all data points in cluster and assigns new centres 
    def update_centres(self, data):
        self.centres = np.array([data[self.cluster_labels == i].mean(axis = 0) for i in range(self.k)])
        return self.centres
    
    # Predict which cluster data point belongs to
    def get_prediction(self, data):
        return self.assign_clusters(data)
    
    # assign all data to 2nd cluster
    def assign_to_random_clusters(self, data):
        self.cluster_labels = np.ones((data.shape))
        
    #  main loop to fit the algorithm       
    def fit_kmeans(self, data):
        #print(self)
        self.centres = self.random_centres(data)
        self.assign_to_random_clusters(data)
        
        # main kmeans loop
        for iter in range(self.max_iter):
            self.cluster_labels = self.assign_clusters(data)
            self.centres = self.update_centres(data) 
            if iter % 5 == 0:
                print("Model Iteration %d " %iter)
        print("Cluster labels:::*********")
        #print(self.cluster_labels)
        #print(type(self.cluster_labels), self.cluster_labels.shape)
        print("Model finished running")
        return self   

In [15]:
from IPython.display import display
def print_cluster(label, n):
    amreviews_sample["cluster_label"] = list(labels)
    for cluster in range(n):
        print("Cluster:", cluster)
        display(amreviews_sample[amreviews_sample.cluster_label == cluster][["review","cluster_label"]].head(5))

In [16]:
# Cluster =5 
n = 5
#Generate 5 random indices 
np.random.seed(45678)
indices = np.random.choice(tf_idf.shape[0], n, replace=False)
# Store the 5 randomly selected reviews as cluster centres
cluster_centres = tf_idf[indices]

In [17]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
km = Kmeans(n, seed= 1234, max_iter=35)
labels = km.fit_kmeans(tf_idf).cluster_labels
print_cluster(labels, n)

Model Iteration 0 
Model Iteration 5 
Model Iteration 10 
Model Iteration 15 
Model Iteration 20 
Model Iteration 25 
Model Iteration 30 
Cluster labels:::*********
Model finished running
Cluster: 0


,review,cluster_label
15918,Nothing special about these sippy cups very functional not a bad investment you may find some cheaper no real complaints,0
126212,"I didn't really begin to use the Piyo Piyo 4-layer milk powder dispenser until my baby became exclusively formula fed. It is one of the greatest inventions for moms! I read in other reviews before purchasing that the stackable screw-on containers were a difficult to screw on and off, I found that to be the case in the beginning but found out that you need to align the layer right on top of the bottom before screwing on.On days when I am not too sure how long I will be out I use all four containers, on days when I'm out only for a couple of hours I fill up only 1 layer and take it with me- having it in my diaper bag just incase really gives me a peace of mind as well. The funnel top makes it very easy to pour into bottles without any mess, I have found that when the bottle is filled to the top with water it can be very hard to pour all that formula powder into it but this would be the case whether you are using this dispenser or not, the trick is to simply use a big bottle opposed to the small bottle.Thus far the dispenser has served me well and I have no real complaints about it. I love it! And it would make a great snack container too when baby get old enough to actually eat them.",0
86781,I bought a barely used pump off of a coworkers wife and I replaced everything but the motor. The medela accessories kit does not come with replacement tubing so this was perfect and so was the price.,0
127474,"We bought this carseat for our second car. We wanted a more ""portable"" choice then our primary car seat. The seat has all the promised features. It is very easy to install and uninstall. We yet have to test how easy it folds and can be carried. It is on the heavy side, so I don't see how it can be carried on the back.Also, it is not as cushy and comfortable as the bigger carseats. But my son stayed in it for 5 hours drive and he didn't complain :)",0
15720,"The Philips AVENT Bottle Brush has a thick cluster of bristles at the end which really help with scrubbing out the bottom of the bottles. I soak my bottles in a large bowl using a combo of vinegar, baking soda, and water after which I use the brush to clean out the bottles, and the result is squeaky clean bottles and parts. The back of the brush has a short groove running down the center which my finger rests on when I'm using it to clean, it's a small detail but I notice that it makes it more comfortable to use. I'm really happy with this purchase!",0


Cluster: 1


,review,cluster_label
184872,"I like the Vichy brand; some time ago, I got hy Puret&eacute; Thermale Fresh Cleansing Gel Cleanser from Amazon Vine, which became my go-to daily cleanser (sometimes I switch it up with other brands just because why not).It works great, and a little goes a long way - for $18 in most places, I get the 6.76oz container that easily lasts me 3 months or more.\n\nI suppose I should have read the description better for this product, because while the set includes a full sized Id&eacute;alia Antioxidant Serum (which I interpret as ""daily moisturizer), the other products are ""deluxe sample sizes"" of Puret&eacute; Thermale 3 -in-1 Cleanser and Id&eacute;alia Night - bizarre because I would have thought the cleanser (a product you use twice a day) would come full sized.\n\nThe tin is beautiful - but when you open it, you see three tiny boxes wrapped in tissue paper thrown inside. There is no packaging whatsoever to keep the items from bouncing all over the place in the massive tin - the tin itself is far larger than the products. I guess it should have been obvious to me again from pictures on Amazon, but I would have thought it would have been secured inside and not just wrapped in tissue.\n\nAfter restraining my disappointment from the massive tin vs tiny products and the ""deluxe trial size"" vs full sized, I tried the product - and they are fine. I use the tiniest dab of the cleanser, one pump of the ""serum"" and then dip the tip of my ring finger in for the night cream. I don't see that this product works better than my previous products - it works about the same.\n\nThe bonus with this set is that it makes for easy travel - and if you are buying for that reason, go for it. It is pretty pricey for a gift set in my opinion at $39 currently on Amazon. I bought a travel-sized cleanser from Whole Foods for about $3, and then a day/night moisturizer from SheaMoisture for about $12 and boom, beauty routine is done, all for well under $20.\n\nIf you want a special set for a person who travels a lot - well, maybe this would work. The products are nice, in an of themselves, even if the samples are small. I might be inclined to repurchase some of the items if I get low on them and the price is right.",1
183591,It was NOT THE COLOR SHOWN NOR WHAT I expected...its a brown/gray,1
169574,"It initially does appear to give your hair a fuller look. However, on closer look, it mainly falls to the scalp. I didn't have problems with sweat or it staining my clothing, etc, but I didn't use it for too long for fear that people would notice that my scalp appeared black. I have a slightly fuller head than typical guys that are looking at this product, but my hair is thin, and my scalp does show in brighter lighting. I was hoping this product could disguise that. In a sense, it does on passing glance or from a distance. I have not had the courage to try it at work.",1
168717,I have used this Jane Iredale product now for 6 years. Once introduced to it I fell in love with the appearance it gives your skin. Does not flake or look drying on the skin. Use with the stamping brush (bought separately) and stipple it onto your face with this brush. It leaves no residue and makes your skin look like porcelain. Just make sure you get the right color shade for your skin. I am fair skin and this works great.,1
162577,"This nail polish is the best! Yes, it is kind of expensive, but it does last and the colors are always fresh and beautiful.",1


Cluster: 2


,review,cluster_label
202439,"Well, I don't really know what else to say.. I ordered them for the guitar, Though I won't be using them for the banjo. (I prefer the sound of clawhammer banjo)The thumb picks fit my thumb perfectly, and the finger picks only needed a tiny squeeze to stay put. They feel kind of alien right now, as it's the first time I've ever wore finger picks, but I'll get used to them eventually :P.",2
4781,"This monitor is decent for the price, I guess. But I think I would have rather spent a little extra to get something a little better. The pro to the monitor is that it picks up the smallest sounds. But the problem is that there's so much static that you can barely hear the sounds that it picks up. There's so much static that it almost completely drowns out my crying baby! If you run the monitor on just the batteries alone ( and unplug the AC adapter), then the monitor is pretty crystal clear. The only problem with that is that the battery life in that monitor is only like 1-2 nights (tops!). So, if you have the money to spend on a new 9V battery every night, or every other night, then I guess this monitor would be okay. But if you have that kind of money, I would suggest that you invest in a monitor that is better quality than this one. I think I am going to return this for something a little better. I'm very disappointed in Fisher Price. They usually have such great quality products. I can't believe they would come out with something this poor of quality. My suggestion: don't [spend] your money on this monitor.",2
197360,The pick holder work as described. I have not used the adhesive to attach the pick holder to my guitar yet but it stays attached to the head piece by sliding it between strings from the tuning pegs on the head piece. It is only sized for a standard size guitar pick so if you use small picks or large picks it will not hold your pick. It works well for standard guitar picks and will hold 5 or 6 picks if you like different thickness picks you can have a variety of picks that will store on the guitar or other pick instrument.,2
115372,Positives:- nice design- not clear - so easy to spot in the crib/floor at night- baby loves it!So-so:- shipped only one pack even though description says two packs. Did receive 2nd pack after complaining about it.- there is only one correct way of putting these in - meaning there is an UP and DOWN sides to these pacifiers. Wish they were &#34;whichever way&#34; like other AVENT pacis.,2
78856,"I ordered this off of amazon due to the reviews it has received. I read the sizing chart and ordered according to the companies recommendations. When it arrived it was way to small. I could not close the front. I also reviewed how it is set up and it basically holds the pumps to your breast, so you don't want it too tight. It could easily form a clogged duct with the amount of pressure if it was too tight. So, I emailed the company and requested a return, I explained that I felt the sizing chart was off. The company accepted my suggustion, said they would take a look at the sizing chart and they accepted my return. I am giving 4 stars not for the product, I would give 3 stars especially for the price. I'm sure this did not nearly cost this much to make, even if it is imported. We design and import organic bibs and the mark up is a little greedy if you ask me, but I believe there is a certain percentage that Amazon needs to take. However, I am giving it 4 stars for customer service. Thanks for accepting my return.",2


Cluster: 3


,review,cluster_label
138288,"I started letting my son play in this walker when he was about 4 months old. He would lean forward because he was still needing support to sit upright, so I started rolling up a towel or blanket and putting it between his chest and the walker. Let me tell you, he just loved it from the moment we put him in! He would just laugh and laugh and push off with his feet like crazy.I think we ejoy watching him in it more than him. I love the brake pads feature which prevent the walker from going over any changes in flooring, like stairs or a door jam. I actually took them off, as our house is only one story, and I am always near to him while he is in it.THe little piano tray is excellent, and has such a variety of sounds and songs that he can make. THey are not too annoying or shrill sounding, either, like so many childrens toys.It was not too hard to put together, although i did need to follow the manual pretty closely, as I could not figure out the pieces on my own. I think it probably took about 20 minutes to get all set up.I like that the music tray is removeable, so it can become a tray for little toys or even snacks later on.I wold highly recommend this toy, as my son seems to like it very much",3
29457,This is a really great little potty chair. My granddaughter loves it. She gets so excited when the music goes off when she &#34;makes potty&#34; and we all applaud. Highly recommend it.,3
47180,"This product is really large (takes up my 4 month olds whole bottom half of her face), really bulky, and really heavy (which makes it tough for her to hold onto or keep in her mouth). I wouldn't recommend it for early teething babies. We may try again in a few months.",3
82572,My baby loves this! He likes all the different textures and colors and will play for ten minutes at a time (that's forever for a six-month-old) with his dragon. I often worry about fabric toys' stitching but despite being gnawed on for months it is still sound.,3
131116,"Our daughter loves this. Yes, it's a little pricey, but it's worth it because we are so much less likely to lose this one versus others (which somehow find themselves rolling into the driveway, under the restaurant table, etc.). Stands up well when you wash the whole thing, which was a big selling point for me. I like this one because a couple kids at her daycare have them but nobody had the pink bear but my girl.",3


Cluster: 4


,review,cluster_label
124534,"These washcloths have beautiful colors. They are not terry cloth and are rather thin, but will do for wiping dirty faces, hands and runny noses. They are small enough to fit several in your diaper bag with no problem.",4
63914,"This warmer has made diaper changes much better for my little one. The only issue I have with this warmer is the lid will pop open (All though I actually have 2 of them, and it does not happen on the other)",4
87727,"Love this for swimsuits, dirty diapers, and as a bag within the diaper bag to fill with just a few diapers, wipes and diaper cream. On a recent 5 hour airplane flight, we just kept this bag under the seat in front of us with changing necessities and a change of clothes for baby. We were able to put the larger diaper bag in the overhead compartment and had a lot more leg room.",4
70018,I LOVE HOW MY BABY BOY LOOKS WITH HIS LITTLE BACKPACK. I CAN'T WAIT FOR HIM TO START USING IT.,4
18696,"This is a very soft sheet at a great price! I was very happy with the look and feel of these sheets. We ordered chocolate brown, and it has a very nice &#34;sheen&#34; to it as well--almost like a sateen sheet. The only downside is that it does not fit around the mattress of our pack-n-play. I measured before purchasing, and it seemed as though it should fit, however, there is a great deal of extra fabric. It almost seems as if it would fit a much more thick mattress (maybe even up to 2-3 inches or so.)",4


In general with 5 clusters, we can see some patterns. Cluster 0 talks about milk bottles and baby cribs. 
Cluster 1 talks about hair products, nail polishes, skin care. Cluster 2 talks about musical instruments and componenets associated with them.
Cluster 3 talks again about baby products , this time about walkers, toys etc. 
Cluster 4 is grooming products for babies which include bedsheets, warmers, washclothes. 
Again, it is not very accurate , but yes, there are visible similarties. 

In [18]:
# Cluster = 10
n = 10
#Generate 3 random indices 
np.random.seed(45678)
indices = np.random.choice(tf_idf.shape[0], n, replace=False)
#indices
# Store the 10 randomly selected reviews as cluster centres
cluster_centres = tf_idf[indices]
km = Kmeans(n, seed= 1234, max_iter=30)
labels = km.fit_kmeans(tf_idf).cluster_labels
print_cluster(labels, n)

Model Iteration 0 
Model Iteration 5 
Model Iteration 10 
Model Iteration 15 
Model Iteration 20 
Model Iteration 25 
Cluster labels:::*********
Model finished running
Cluster: 0


,review,cluster_label
15918,Nothing special about these sippy cups very functional not a bad investment you may find some cheaper no real complaints,0
54295,"Stokke makes beautiful high chair options for those of us who like more sleek, furniture style baby needs. It fits beautifully in my dining room with my adult furniture. it is a more expensive option but think of all the uses you get as the years go on, even as an adult seat option. that makes the price completely affordable when you compare how long you use a standard high chair.",0
32180,"Not sure how these are ""soft"" tip. They work just fine, clean just fine in the dishwasher (top rack), but they're just not ""soft."" My kiddo (8 months) does well with silicone tips, but these are hard plastic, so he can't empty them quite as well and occasionally hurts his gums on them when he chomps down. (He tends to chomp down on the silicone spoons - uses them like a teether sometimes.)",0
107612,"Make sure you clean your door really well before applying the item. I just kind of wiped the door really quickly and stuck it on there and it fell off...still had some dust on the door I guess. Other than that, now that it's stuck on better, works fine.",0
5043,Excellent product. Simple to install and effectively ends all worries about my son accidentally choking on the rubber end of a typical door stop.,0


Cluster: 1


,review,cluster_label
202439,"Well, I don't really know what else to say.. I ordered them for the guitar, Though I won't be using them for the banjo. (I prefer the sound of clawhammer banjo)The thumb picks fit my thumb perfectly, and the finger picks only needed a tiny squeeze to stay put. They feel kind of alien right now, as it's the first time I've ever wore finger picks, but I'll get used to them eventually :P.",1
183591,It was NOT THE COLOR SHOWN NOR WHAT I expected...its a brown/gray,1
197360,The pick holder work as described. I have not used the adhesive to attach the pick holder to my guitar yet but it stays attached to the head piece by sliding it between strings from the tuning pegs on the head piece. It is only sized for a standard size guitar pick so if you use small picks or large picks it will not hold your pick. It works well for standard guitar picks and will hold 5 or 6 picks if you like different thickness picks you can have a variety of picks that will store on the guitar or other pick instrument.,1
196978,"I have learned out of experience that when you first pick up guitar, it is a bad idea to practice on a cheap instrument. Cheap $50-100 instruments do not play accurately, and you would be more likely give up. This, on the other hand, is NOT a low-quality instrument. Takamine makes many excellent instruments, this included. It plays very loud and clear, zero buzzing. If you are looking for a practice guitar for either yourself or someone else, look no further. I can guarantee that you will not find a guitar this magnificent for such an affordable price. Note: the package does not include a gig back or case, So I bought this one here, that fits the guitar perfectly:Musician's Gear Deluxe Dreadnought Case Black (Black)",1
169574,"It initially does appear to give your hair a fuller look. However, on closer look, it mainly falls to the scalp. I didn't have problems with sweat or it staining my clothing, etc, but I didn't use it for too long for fear that people would notice that my scalp appeared black. I have a slightly fuller head than typical guys that are looking at this product, but my hair is thin, and my scalp does show in brighter lighting. I was hoping this product could disguise that. In a sense, it does on passing glance or from a distance. I have not had the courage to try it at work.",1


Cluster: 2


...

,review,cluster_label
124534,"These washcloths have beautiful colors. They are not terry cloth and are rather thin, but will do for wiping dirty faces, hands and runny noses. They are small enough to fit several in your diaper bag with no problem.",7
63914,"This warmer has made diaper changes much better for my little one. The only issue I have with this warmer is the lid will pop open (All though I actually have 2 of them, and it does not happen on the other)",7
87727,"Love this for swimsuits, dirty diapers, and as a bag within the diaper bag to fill with just a few diapers, wipes and diaper cream. On a recent 5 hour airplane flight, we just kept this bag under the seat in front of us with changing necessities and a change of clothes for baby. We were able to put the larger diaper bag in the overhead compartment and had a lot more leg room.",7
123143,"We have a couple and we hang them from the corner of our changing table so they are very handy for diaper changes. Hasn't leaked or anything yet. The yellow is very bright! We like pumpkin color better. As other reviewers have mentioned, the bag does not stand up, as the photo is a little misleading.",7
129899,"LEAK like crazy! I loved them when we were using them because I was so excited to use cloth diapers and so excited about my baby that I didn't mind baby pee on me. I thought the leaks were normal. Now that we are in regular sized diapers I know that the leaks were not normal. The seams around the legs seem like they are unfinished so that the inner fabric touches whatever baby is wearing or sitting on. So even if the diaper doesn't drip pee it will always seep it. Some days (and often) baby went through a new change of clothes with every diaper change. The first time I just used one outfit all day long I thought it was a miracle. But no, thats how the diapers should work! I stripped the diapers and everything, but they always leaked. I currently use bigger size GroVia diapers along with several other brands and don't have a leak problem. It is obviously specific to the make and fit of the NEWBORN diapers. Horrible.We only bought GroVia brand for the newborn sized diapers. We currently use several brands of one-size diapers. I HIGHLY recommend buying several brands of both newborn and one-size diapers. The one-size can be too huge on a newborn. Its nice to have something that fits better for the first month or two. But GroVia newborn diapers leaked like crazy for us. I'll try them again with baby two, but am definitely buying another brand's newborn diapers next time!",7


Cluster: 8


review  \
10176   This is terrific!  Couldn't live without it!  If you empty it everyday there is no smell. Easy to use. Easy to empty. Easy to refill.  It is just great!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Cluster: 9


review  \
184872  I like the Vichy brand; some time ago, I got hy Puret&eacute; Thermale Fresh Cleansing Gel Cleanser from Amazon Vine, which became my go-to daily cleanser (sometimes I switch it up with other brands just because why not).It works great, and a little goes a long way - for $18 in most places, I get the 6.76oz container that easily lasts me 3 months or more.\n\nI suppose I should have read the description better for this product, because while the set includes a full sized Id&eacute;alia Antioxidant Serum (which I interpret as "daily moisturizer), the other products are "deluxe sample sizes" of Puret&eacute; Thermale 3 -in-1 Cleanser and Id&eacute;alia Night - bizarre because I would have thought the cleanser (a product you use twice a day) would come full sized.\n\nThe tin is beautiful - but when you open it, you see three tiny boxes wrapped in tissue paper thrown inside. There is no packaging whatsoever to keep the items from bouncing all over the place in the massive tin - the tin itself is far larger than the products. I guess it should have been obvious to me again from pictures on Amazon, but I would have thought it would have been secured inside and not just wrapped in tissue.\n\nAfter restraining my disappointment from the massive tin vs tiny products and the "deluxe trial size" vs full sized, I tried the product - and they are fine. I use the tiniest dab of the cleanser, one pump of the "serum" and then dip the tip of my ring finger in for the night cream. I don't see that this product works better than my previous products - it works about the same.\n\nThe bonus with this set is that it makes for easy travel - and if you are buying for that reason, go for it. It is pretty pricey for a gift set in my opinion at $39 currently on Amazon. I bought a travel-sized cleanser from Whole Foods for about $3, and then a day/night moisturizer from SheaMoisture for about $12 and boom, beauty routine is done, all for well under $20.\n\nIf you want a special set for a person who travels a lot - well, maybe this would work. The products are nice, in an of themselves, even if the samples are small. I might be inclined to repurchase some of the items if I get low on them and the price is right.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [18]:
# Assign number of clusters to variables
n = 15
#Generate 15 random indices 
np.random.seed(45678)
indices = np.random.choice(tf_idf.shape[0], n, replace=False)
#indices
# Store the 15 randomly selected reviews as cluster centres
cluster_centres = tf_idf[indices]
km = Kmeans(n, seed= 1234, max_iter=35)
labels = km.fit_kmeans(tf_idf).cluster_labels
print_cluster(labels, n)

Model Iteration 0 
Model Iteration 5 
Model Iteration 10 
Model Iteration 15 
Model Iteration 20 
Model Iteration 25 
Model Iteration 30 
Cluster labels:::*********
Model finished running
Cluster: 0


,review,cluster_label
15918,Nothing special about these sippy cups very functional not a bad investment you may find some cheaper no real complaints,0
124308,"i love my nuk cup its a great design. The bottle is not to wide the spout is spill proof but still has a good flow.plus thay have cute designs. the only con is that the replacement spout is expensive, i also have the munchkins sippy cup which was also great but those spots tear quicker and are cheap looking.so i prefer nuk!",0
32180,"Not sure how these are ""soft"" tip. They work just fine, clean just fine in the dishwasher (top rack), but they're just not ""soft."" My kiddo (8 months) does well with silicone tips, but these are hard plastic, so he can't empty them quite as well and occasionally hurts his gums on them when he chomps down. (He tends to chomp down on the silicone spoons - uses them like a teether sometimes.)",0
91120,I love this. I make my own baby food and this tray works great. The frozen cubes pop out so easily. Its awesome.,0
90566,"The handles on these are great for a toddler, very easy to grip. And the fork and spoon are shaped well for holding the food, even for an unsteady new utensil user. But they are a little big for my toddler's mouth. She can use them, but she'd have an easier time if the part that goes in the mouth were a bit smaller.",0


Cluster: 1


,review,cluster_label
169574,"It initially does appear to give your hair a fuller look. However, on closer look, it mainly falls to the scalp. I didn't have problems with sweat or it staining my clothing, etc, but I didn't use it for too long for fear that people would notice that my scalp appeared black. I have a slightly fuller head than typical guys that are looking at this product, but my hair is thin, and my scalp does show in brighter lighting. I was hoping this product could disguise that. In a sense, it does on passing glance or from a distance. I have not had the courage to try it at work.",1
190859,"Has floral scents with a dash of nutmeg and spice. Of course, the smell varies for everyone but I found it a little too overpowering for my tastes. You do get quite a bit of the liquid, and I've taken a photo next to a 12oz can of soda for comparison purposes. Overall, not bad for the price.",1
169022,"CHI Silk Infusion is a great hair product. It left my hair very soft and shiny. Plus, it smells great. I like that it doesn't take a huge amount of product to get through my thick, wavy hair either.\n\nUnfortunately, it has been very rainy and humid here so I haven't been able to see the full extent of what this product can do. However, I have high hopes for this. I'll definitely use this all summer long to keep frizz at bay.",1
199368,The Chorus is subtle... but it seems lacking to me. It is subtle but really weak compared to the classic rich ibanez chorus i have.,1
189460,"Gentlemen, if your stylist reaches for this stuff- RUN! It's left me with the most horrid smelling hair, like someone dumped a bottle of old lady perfume on me. It is way too strong and fragrant, very feminine too. It's lasted for days now even after I shampoo. Holy hell.",1


Cluster: 2


...

,review,cluster_label
86781,I bought a barely used pump off of a coworkers wife and I replaced everything but the motor. The medela accessories kit does not come with replacement tubing so this was perfect and so was the price.,12
75784,"Nothing like a hands-free bustier to make you feel like a dairy cow! I hook up my double pump backpack and go about my day. I will occassionally catch myself in the mirror, and I will kindly regard my reflection with a &#34;moo.&#34;",12
13474,"I waited four months before purchasing a hand pump, and wish I had done so sooner. My insurance covered the full cost of the Medela Pump In Style, which I use daily while at work. While you can't beat an electric double pump for efficiency, sometimes it's really nice to have a hand pump. I chose this one simply because the valves, bottles, and breast shields are interchangeable with the ones for my electric pump, and it works just fine. The pumping mechanism itself seems well designed so it's pretty comfortable to pump, even for my relatively small hands. It's really easy to clean since the pumping part itself never comes in contact with milk.Having a great sleeper, I've struggled with engorgement, especially at night, and this hand pump has quickly become essential. It's so much quieter and easier to set up than my electric pump that it's perfect for quick night-time relief. The suction seems gentler than the electric pump so is more comfortable, but takes longer to fully drain the breast. Since I'm not looking for full drainage or stimulation at night, this pump is perfect. I've also used it while a passenger in a car and in various public restrooms when it's bulky electric big sister would be inconvenient. Everything comes apart and is small enough to fit in a zip-lock sandwich bag (except the bottle).",12
3354,"You absolutely cannot wear these out in public! This is for home use only! They do not match the contour of the breast at all so there's a totally weird bulge on your boob when you wear them. So with that being said, if you do just wear these at home, they can be great! If you have really raw nipples they offer a much needed relief and the ability to get air. I use them at night so that they catch anything that I leak, and I can then just pour the milk into the baby's mouth via a little spout on the edge. I find that I go the longest without nursing through the night, so this offers a great way to never waste a drop!",12
1084,"Awesome product, a little loud but overall does the job, we rented a machine from the hospital tht did the same for twice the money we paid for this, after realizing we needed additional time and they required additional fees, we opted to purchase this and keep rather than spend it on rental, great choice we made, highly recommended.",12


Cluster: 13


,review,cluster_label
183591,It was NOT THE COLOR SHOWN NOR WHAT I expected...its a brown/gray,13
162577,"This nail polish is the best! Yes, it is kind of expensive, but it does last and the colors are always fresh and beautiful.",13
161402,Not too soft just right the color is perfect this is exactly what I was looking for.,13
183624,"Does not shout ""look at my toes"" like so many of the contemporary colors. This ""old-fashioned"" color is not available in local stores. I love it - not too pink, not too red. Just right.",13
175687,"It has a tiny bit of sparkle which I would normally not like at all, but this is very light sparkle, can barely see it. Love the color",13


Cluster: 14


,review,cluster_label
107612,"Make sure you clean your door really well before applying the item. I just kind of wiped the door really quickly and stuck it on there and it fell off...still had some dust on the door I guess. Other than that, now that it's stuck on better, works fine.",14
5043,Excellent product. Simple to install and effectively ends all worries about my son accidentally choking on the rubber end of a typical door stop.,14
110278,"My son was obsessed with our exterior door baby locks and kept pulling them off as soon as he saw that one was loose or unlocked. These locks work great, as there is no external hardware. Now my son simply tries to open a cabinet and walks away when he realizes that it doesn't open. There are two swithes on the lock that keep them locked or unlocked. That works well when you don't want your cabinet to be locked at all times or if someone needs to babyproof their house on occasion, but not all of the time. I just stick the key to my fridge (magnet) so I won't lose it. It does take some getting used to, as you can't see the lock from the outside of the cabinet to remind you it is there, but it sure does beat having to mess with external locks while trying to get into your cabinet. I also like that the cabinet won't open the slightest bit while being pulled on, which keeps my son's fingers from getting pinched. I would highly recommend this product.",14
123686,This product is easy to install and easy to use.Took 30 seconds to install and takes a couple of seconds to go from lock to unlock.I feel comfortable now with my little one in the kitchen with the oven on.,14
204070,"Yes, I bought 3 and they fit all my guitars, mandolins, ukulele, bass.Very easy to install on the wall.",14


In [19]:
# Cluster = 20
n = 20
#Generate 5 random indices 
np.random.seed(45678)
indices = np.random.choice(tf_idf.shape[0], n, replace=False)
#indices
# Store the 5 randomly selected reviews as cluster centres
cluster_centres = tf_idf[indices]
km = Kmeans(n, seed= 1234, max_iter=35)
labels = km.fit_kmeans(tf_idf).cluster_labels
print_cluster(labels, n)

Model Iteration 0 
Model Iteration 5 
Model Iteration 10 
Model Iteration 15 
Model Iteration 20 
Model Iteration 25 
Model Iteration 30 
Cluster labels:::*********
Model finished running
Cluster: 0


,review,cluster_label
130719,"I love how this seat fits snuggly in the round toilet seat we have. I also love the splash guard. It is deep and has inner walls to direct the pee. I haven't put it into use yet, but it was exactly what I was looking for and for the right price!",0
91120,I love this. I make my own baby food and this tray works great. The frozen cubes pop out so easily. Its awesome.,0
90566,"The handles on these are great for a toddler, very easy to grip. And the fork and spoon are shaped well for holding the food, even for an unsteady new utensil user. But they are a little big for my toddler's mouth. She can use them, but she'd have an easier time if the part that goes in the mouth were a bit smaller.",0
9697,"The product works well, in theory. Locked the food in place with no problem. Easy to grip. But the mesh is super thick and large. My little girl, who loves food and puts everything in her mouth, didn't want to put this in her mouth. She tried it, made an awful face and refused to put it near her face again. I introduced at 6 months, and then every few weeks after that. I've since given up.I don't blame her. I tried to eat an apple slice through it. I got hardly any juice and the feel of the mesh in my mouth was gross. Very unappetizing.Maybe if they had a thinner mesh??",0
42103,"I was a little skeptical ordering this due to the listed dimensions, but it stretches to accommodate an amazingly large amount of toys.We have quite a bit of weight in ours and it sticks very well. Occasionally one of the suction cups will pop off, but is immediately and easily reapplied by wetting the cup a little.",0


Cluster: 1


,review,cluster_label
169574,"It initially does appear to give your hair a fuller look. However, on closer look, it mainly falls to the scalp. I didn't have problems with sweat or it staining my clothing, etc, but I didn't use it for too long for fear that people would notice that my scalp appeared black. I have a slightly fuller head than typical guys that are looking at this product, but my hair is thin, and my scalp does show in brighter lighting. I was hoping this product could disguise that. In a sense, it does on passing glance or from a distance. I have not had the courage to try it at work.",1
190859,"Has floral scents with a dash of nutmeg and spice. Of course, the smell varies for everyone but I found it a little too overpowering for my tastes. You do get quite a bit of the liquid, and I've taken a photo next to a 12oz can of soda for comparison purposes. Overall, not bad for the price.",1
169022,"CHI Silk Infusion is a great hair product. It left my hair very soft and shiny. Plus, it smells great. I like that it doesn't take a huge amount of product to get through my thick, wavy hair either.\n\nUnfortunately, it has been very rainy and humid here so I haven't been able to see the full extent of what this product can do. However, I have high hopes for this. I'll definitely use this all summer long to keep frizz at bay.",1
199368,The Chorus is subtle... but it seems lacking to me. It is subtle but really weak compared to the classic rich ibanez chorus i have.,1
189460,"Gentlemen, if your stylist reaches for this stuff- RUN! It's left me with the most horrid smelling hair, like someone dumped a bottle of old lady perfume on me. It is way too strong and fragrant, very feminine too. It's lasted for days now even after I shampoo. Holy hell.",1


Cluster: 2


...

,review,cluster_label
183591,It was NOT THE COLOR SHOWN NOR WHAT I expected...its a brown/gray,17
70018,I LOVE HOW MY BABY BOY LOOKS WITH HIS LITTLE BACKPACK. I CAN'T WAIT FOR HIM TO START USING IT.,17
123143,"We have a couple and we hang them from the corner of our changing table so they are very handy for diaper changes. Hasn't leaked or anything yet. The yellow is very bright! We like pumpkin color better. As other reviewers have mentioned, the bag does not stand up, as the photo is a little misleading.",17
161402,Not too soft just right the color is perfect this is exactly what I was looking for.,17
192666,"After trying several Vita Liberata tanning products, the last one I used had me raving with joy (that was the Fabulous Self Tanning Gradual Lotion - the best self tanner I've ever used.) I should've stopped while I was ahead, because the Body Blur Instant HD was a bit of a letdown.\n\nDon't get me wrong. It gives a decent color, particularly on the legs (my personal bete noire), although this was probably a bit dark for me. I found the application streaky, no matter what I did. And the idea of putting this on my face? Oh gawd, no!\n\nThe real problem with this is that it rubs off. I had this on my legs, hours before I went to bed, and it didn't feel tacky. In the morning I woke up to sheets with tan-colored smears, the product embeded in the fibers. I washed them immediately, but the stains remain. Well, at least I found something where this product sticks, cause it sure wasn't my legs!\n\nI wish I could recommend this more highly, but it's a disappointment. Fortunately, the company's Fabulous Self Tanning Gradual Lotion IS available on Amazon. Drop this, and get that, you love it!",17


Cluster: 18


,review,cluster_label
16398,"The SwaddleMe with microfleece is great, warm (not sure to use during really hot months) but the best part is the really strong velcro, very different than the other SwaddleMes. I highly recommend it, and it's great that it comes in this large size!",18
93387,"Good value, but doesn't always stay on the pillow, especially if my toddler has an active night of tossing and turning.",18
22672,"Love love love love it! thanks to our pediatrician for recommending it we get an excellent night sleep (baby sleeps 9-10hrs since 1mo (when we bought the blanket), now at 4mo-she wakes up to eat twice a night but falls back to sleep as soon as she finished eating). our baby sleeps in it every night-no exceptions. She even calms down when she sees the blanket or if she is fussy she stops once swaddled. My Mom (hard core old-fashioned skeptic) was so surprised to see this work, since then she is also recommending it to everyone. If I could give it more stars I would, I absolutely love it. I'm keeping this to use it for my future kids, its just amazing.Thank you Miracle Blanket you are truly a miracle.",18
132263,This makes swaddling very quick and easy to do! It's nice that it is designed so you can do middle of the night diaper changes without unswaddling your baby's arms. I really think my babies did sleep more soundly for longer when swaddled since they didn't startle as much. This product kept my baby swaddled better than when I'd swaddle her with a regular baby blanket since my second child was a real wiggle worm when she slept. These are definitely a must have for new parents!,18
38884,"I don't know what it is about this brand, but it is by far my daughter's favorite. We had received one as a gift when she was born and now that she's 8 months old, she is attached. We had to find the brand and order another one so we can always have one ready to go. My only advice is to buy at least 2 of these things - you will NOT want to find out what happens when one is missing and your little one needs to go to sleep!Definitely plan on buying these for our friends as gifts in the future - something about this brand is magic!",18


Cluster: 19


,review,cluster_label
32180,"Not sure how these are ""soft"" tip. They work just fine, clean just fine in the dishwasher (top rack), but they're just not ""soft."" My kiddo (8 months) does well with silicone tips, but these are hard plastic, so he can't empty them quite as well and occasionally hurts his gums on them when he chomps down. (He tends to chomp down on the silicone spoons - uses them like a teether sometimes.)",19
118455,"Terrible brushes, not only expensive but they broke after 2 months of usage ... The brush part broke off the handle part making it basically useless, i had to throw it away .... Very disappointed.",19
31224,"This arrived today and I haven't used it yet, but wanted to post a review. A couple other reviewers commented on the non-slip/non-skid bottom being great. Well... They must have reviewed the wrong item, because this is NOT non-slip at all. I bought it to place on the vanity in our master bathroom, and will need to be careful so it doesn't slide (can't secure it to the vanity top, so was really counting on the non-slip).Other than that, this seems nice for the price, I am happy with the 4 sides (another big factor in choosing this changing pad) and happy with the size as we needed a slightly shorter pad.",19
34152,"Does its job, and it's the cutest thing, but it comes unclipped on the regular which is frustrating... Other than that, it's golden.",19
67544,"I have probably a good dozen binkies around my house. And at any one time, I can guarantee you that I don't know where at least half of them are. My carpet is sage green, so trying to find a dropped binky in the middle of the night just doesn't happen. But I always know where my wubbanub is! We have the cat and the frog - one for our girl and one for our boy. Not only does it help us to easily find the binky, but it also helps the binky to stay more upright when it drops, AND it helps the binky to stay in our babies' mouths. If they are upright, the added weight may help it fall out easier if they aren't sucking aggressively, but when sitting or laying down, it's just awesome.I love these! I don't know what I did without these with my other two kids!",19


When the number of clusters is increased to 10,15. With the number of cluster incrased to 20, it starts to blur the lines between clusters. 

In [19]:
# bigger sample
np.random.seed(1867)
amreviews_sample = amreviews_mod.sample(35000)
X = vectorizer.fit_transform(amreviews_sample.review.values)
tf_idf = get_tfidf_matrix(X)
print(tf_idf.shape)

(35000, 34293)


In [20]:
# Assign number of clusters to variables
n = 15
#Generate random indices 
np.random.seed(45678)
indices = np.random.choice(tf_idf.shape[0], n, replace=False)
#indices
# Store the 5 randomly selected reviews as cluster centres
cluster_centres = tf_idf[indices]
km = Kmeans(n, seed= 1234, max_iter=30)
labels = km.fit_kmeans(tf_idf).cluster_labels
print_cluster(labels, n)

Model Iteration 0 
Model Iteration 5 
Model Iteration 10 
Model Iteration 15 
Model Iteration 20 
Model Iteration 25 
Cluster labels:::*********
Model finished running
Cluster: 0


,review,cluster_label
202439,"Well, I don't really know what else to say.. I ordered them for the guitar, Though I won't be using them for the banjo. (I prefer the sound of clawhammer banjo)The thumb picks fit my thumb perfectly, and the finger picks only needed a tiny squeeze to stay put. They feel kind of alien right now, as it's the first time I've ever wore finger picks, but I'll get used to them eventually :P.",0
4781,"This monitor is decent for the price, I guess. But I think I would have rather spent a little extra to get something a little better. The pro to the monitor is that it picks up the smallest sounds. But the problem is that there's so much static that you can barely hear the sounds that it picks up. There's so much static that it almost completely drowns out my crying baby! If you run the monitor on just the batteries alone ( and unplug the AC adapter), then the monitor is pretty crystal clear. The only problem with that is that the battery life in that monitor is only like 1-2 nights (tops!). So, if you have the money to spend on a new 9V battery every night, or every other night, then I guess this monitor would be okay. But if you have that kind of money, I would suggest that you invest in a monitor that is better quality than this one. I think I am going to return this for something a little better. I'm very disappointed in Fisher Price. They usually have such great quality products. I can't believe they would come out with something this poor of quality. My suggestion: don't [spend] your money on this monitor.",0
197360,The pick holder work as described. I have not used the adhesive to attach the pick holder to my guitar yet but it stays attached to the head piece by sliding it between strings from the tuning pegs on the head piece. It is only sized for a standard size guitar pick so if you use small picks or large picks it will not hold your pick. It works well for standard guitar picks and will hold 5 or 6 picks if you like different thickness picks you can have a variety of picks that will store on the guitar or other pick instrument.,0
133169,use every single day. SUPERB QUALITY :),0
196978,"I have learned out of experience that when you first pick up guitar, it is a bad idea to practice on a cheap instrument. Cheap $50-100 instruments do not play accurately, and you would be more likely give up. This, on the other hand, is NOT a low-quality instrument. Takamine makes many excellent instruments, this included. It plays very loud and clear, zero buzzing. If you are looking for a practice guitar for either yourself or someone else, look no further. I can guarantee that you will not find a guitar this magnificent for such an affordable price. Note: the package does not include a gig back or case, So I bought this one here, that fits the guitar perfectly:Musician's Gear Deluxe Dreadnought Case Black (Black)",0


Cluster: 1


,review,cluster_label
18696,"This is a very soft sheet at a great price! I was very happy with the look and feel of these sheets. We ordered chocolate brown, and it has a very nice &#34;sheen&#34; to it as well--almost like a sateen sheet. The only downside is that it does not fit around the mattress of our pack-n-play. I measured before purchasing, and it seemed as though it should fit, however, there is a great deal of extra fabric. It almost seems as if it would fit a much more thick mattress (maybe even up to 2-3 inches or so.)",1
35120,"These actually fit my graco pack n play. Walmart brand did not. They are very thin though (which is why i gave it 4 stars), but enough that my son isn't just sleeping on the pack n play padding.These actually fit my graco pack n play. Walmart brand did not. They are very thin though, but enough that my son isn't just sleeping on the pack n play padding.",1
16398,"The SwaddleMe with microfleece is great, warm (not sure to use during really hot months) but the best part is the really strong velcro, very different than the other SwaddleMes. I highly recommend it, and it's great that it comes in this large size!",1
31224,"This arrived today and I haven't used it yet, but wanted to post a review. A couple other reviewers commented on the non-slip/non-skid bottom being great. Well... They must have reviewed the wrong item, because this is NOT non-slip at all. I bought it to place on the vanity in our master bathroom, and will need to be careful so it doesn't slide (can't secure it to the vanity top, so was really counting on the non-slip).Other than that, this seems nice for the price, I am happy with the 4 sides (another big factor in choosing this changing pad) and happy with the size as we needed a slightly shorter pad.",1
93387,"Good value, but doesn't always stay on the pillow, especially if my toddler has an active night of tossing and turning.",1


Cluster: 2


...

,review,cluster_label
15918,Nothing special about these sippy cups very functional not a bad investment you may find some cheaper no real complaints,12
126212,"I didn't really begin to use the Piyo Piyo 4-layer milk powder dispenser until my baby became exclusively formula fed. It is one of the greatest inventions for moms! I read in other reviews before purchasing that the stackable screw-on containers were a difficult to screw on and off, I found that to be the case in the beginning but found out that you need to align the layer right on top of the bottom before screwing on.On days when I am not too sure how long I will be out I use all four containers, on days when I'm out only for a couple of hours I fill up only 1 layer and take it with me- having it in my diaper bag just incase really gives me a peace of mind as well. The funnel top makes it very easy to pour into bottles without any mess, I have found that when the bottle is filled to the top with water it can be very hard to pour all that formula powder into it but this would be the case whether you are using this dispenser or not, the trick is to simply use a big bottle opposed to the small bottle.Thus far the dispenser has served me well and I have no real complaints about it. I love it! And it would make a great snack container too when baby get old enough to actually eat them.",12
15720,"The Philips AVENT Bottle Brush has a thick cluster of bristles at the end which really help with scrubbing out the bottom of the bottles. I soak my bottles in a large bowl using a combo of vinegar, baking soda, and water after which I use the brush to clean out the bottles, and the result is squeaky clean bottles and parts. The back of the brush has a short groove running down the center which my finger rests on when I'm using it to clean, it's a small detail but I notice that it makes it more comfortable to use. I'm really happy with this purchase!",12
124308,"i love my nuk cup its a great design. The bottle is not to wide the spout is spill proof but still has a good flow.plus thay have cute designs. the only con is that the replacement spout is expensive, i also have the munchkins sippy cup which was also great but those spots tear quicker and are cheap looking.so i prefer nuk!",12
70779,"We only use AVENT bottles in our home. I really like them, except that they leak! I have read every review possible attempting to find the secret to prevent leaking (I have yet to find a solution). That being said, I will absolutely use them with any additional children I have.Reasons I like AVENT bottles:- Easy to clean. I've hand washed them and used the dishwasher - depending on the space in my dishwasher. The bottles are perfectly safe in the dishwasher, they do tend to become slightly discolored (a light tan-brown). I'm fairly certain that the instructions even say that this happens.- Wide mouth. I like that when pouring formula into the bottle it is wide enough to accommodate the scoop or formula dispenser. I've been told that many other brands are not easy to use with formula because the mouth is very narrow causing a lot of formula to spill.- Price. These bottles are not the cheapest but are much better priced than many other popular brands. I feel as though they are priced in the middle.- Interchangeable parts. I like that I can get these bottles and use them with the sippy handles and sippy spouts to turn them into sippy cups. I also used these for milk/formula storage with the storage disks (which go in the place of the nipple). One bottle can be so many different things for just a small additional price.CONS:- Again, these bottles DO leak. I've both put the bottles together when wet and dry and have noticed no difference. Additionally, I've even mixed the formula and poured into this bottle and it still leaked. I like these bottles for so many other reasons so the leaking isn't a deal-breaker (also, my baby tends to make a huge mess anyway so leaking doesn't really matter).UPDATE:- 

Cluster: 13


,review,cluster_label
172864,"I'm a woman in my 40s who has switched to shave soaps & a brush after years of canned foam. The Proraso green soap/croap is my favorite, and the white is very similar. Both are more of a ""croap"" (cross between hard cream & soap) rather than a straight up hard puck of soap. It lathers crazy easily with a wet badger or boar brush & the ""Marco method"" (if you're new to wet shaving or having trouble making lather, google it - you won't regret it), and provides a good, close, irritation-free shave. I lather in the plastic Proraso bowl directly on the soap, then go straight to leg. I have also done a separate bowl lather with this, and it worked nicely. After use I leave the lid off for a couple of hours to let the soap dry somewhat (I shave in the shower & store my soap there, so there's a lot of water) and then store it with the lid on. The white has a slight menthol feel, but considerably less than the green. In my opinion, the white has a light scent of roses & fresh cut grass. It's very light and clean, and I think it could be considered the most ""feminine"" between the green, red & white Proraso soaps. I had issues with irritation from the red Proraso (disappointing, since I loved the scent), but neither the green or the white cause any problems for me. Overall I am extremely pleased with this soap and it will definitely be in my regular rotation.",13
171565,"This stuff is kind thin and doesn't seem to give as smooth a shave as the regular proraso cream. The lighter smell is ok but it just doesn't lather well. I have sensitive skin and love Proraso's regular line of products but this white label ""sensitive skin"" line is too sensitive for me. I'll stick to the regular proraso line.\n\nSHAVING- let me save you years of trial and error: take a warm shower & wash your face with billy jealousy exfoliating facial cleanser, shave afterwards with a $50 Merkur safety razor handle with good blades (save $100's on mach 3 refills!), use a $20 Tweezerman badger hair shaving brush to apply taylor of old street bond (or proraso) shaving cream (you can let it set in a minute), shave WITH the grain DOWN, SLOWLY, SOFTLY (don't press hard- let the weight of the handle and sharp blade do the cutting), lastly rub on a fingertip of proraso pre/post shave cream while your face is still wet and let it dry! Say goodbye to razor bumps and ingrown hairs FOREVER! Trust me- I've tried dozens of pricey face washes, shaving creams, and after shaves over the last 4 years. If you don't want to commit so much money yet- order the pre/post shave proraso cream online, buy proraso shave cream online or at bath and body works (under their C.O. Bigelow label) then get the cheap single blade disposables (mach 3's are too aggressive) at a grocery store and you'll notice a difference overnight.",13
172703,I have very sensitive skin. Over the years I have tried many different things to help. This is the second order of Proraso Shaving Soap I have made and I do not anticipate changing. The product lasts a very long time and does not have an overwhelming scent. This was good for me as I also don't do well with perfumes or lotions that have smells. This shaving soap has a clean scent but very faint. The most important thing is that this soap has improved my skin tremendously. I use this shaving soap and switched to a DE razor. Although my skin hasn't been perfect it has been a tremendous amount better using this combination than anything else I have tried through the years. I am very happy with this purchase.,13
172793,I have very sensitive skin. Over the years I have tried many different things to help. This is the second order of Proraso Shaving Soap I have made and I do not anticipate changing. The product lasts a very long time and does not have an overwhelming scent. This was good for me as I also don't do well with perfumes or lotions that have smells. This shaving soap has a clean scent but very faint. The most important thing i

Cluster: 14


review  \
184872  I like the Vichy brand; some time ago, I got hy Puret&eacute; Thermale Fresh Cleansing Gel Cleanser from Amazon Vine, which became my go-to daily cleanser (sometimes I switch it up with other brands just because why not).It works great, and a little goes a long way - for $18 in most places, I get the 6.76oz container that easily lasts me 3 months or more.\n\nI suppose I should have read the description better for this product, because while the set includes a full sized Id&eacute;alia Antioxidant Serum (which I interpret as "daily moisturizer), the other products are "deluxe sample sizes" of Puret&eacute; Thermale 3 -in-1 Cleanser and Id&eacute;alia Night - bizarre because I would have thought the cleanser (a product you use twice a day) would come full sized.\n\nThe tin is beautiful - but when you open it, you see three tiny boxes wrapped in tissue paper thrown inside. There is no packaging whatsoever to keep the items from bouncing all over the place in the massive tin - the tin itself is far larger than the products. I guess it should have been obvious to me again from pictures on Amazon, but I would have thought it would have been secured inside and not just wrapped in tissue.\n\nAfter restraining my disappointment from the massive tin vs tiny products and the "deluxe trial size" vs full sized, I tried the product - and they are fine. I use the tiniest dab of the cleanser, one pump of the "serum" and then dip the tip of my ring finger in for the night cream. I don't see that this product works better than my previous products - it works about the same.\n\nThe bonus with this set is that it makes for easy travel - and if you are buying for that reason, go for it. It is pretty pricey for a gift set in my opinion at $39 currently on Amazon. I bought a travel-sized cleanser from Whole Foods for about $3, and then a day/night moisturizer from SheaMoisture for about $12 and boom, beauty routine is done, all for well under $20.\n\nIf you want a special set for a person who travels a lot - well, maybe this would work. The products are nice, in an of themselves, even if the samples are small. I might be inclined to repurchase some of the items if I get low on them and the price is right.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

The clustering works pretty well when the cluster number was increased. There were 4-5 distinct clusters, namely 
- baby products like breast pumps, baby milk
- hair products 
- guitar components/guitar 
- baby cribs, walker 
- lipsticks.

k = 20 was very interesting, as for our sample set, it started to mislabel attributes that it correctly guessed before! 